<a href="https://colab.research.google.com/github/Vivism/Artskuhl-Gallery-Search/blob/master/Artsk%C3%BChl_Gallery_Search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Artskühl Gallery Search

In [0]:
rm -rf sample_data

#### Install TFP2

In [2]:
!git clone https://github.com/googlecodelabs/tensorflow-for-poets-2
!mkdir tfp2
!mv tensorflow-for-poets-2/scripts/* tfp2
!rm -rf tensorflow-for-poets-2

Cloning into 'tensorflow-for-poets-2'...
remote: Enumerating objects: 417, done.
remote: Total 417 (delta 0), reused 0 (delta 0), pack-reused 417
Receiving objects: 100% (417/417), 33.97 MiB | 27.45 MiB/s, done.
Resolving deltas: 100% (155/155), done.


#### Install Graph

In [0]:
!mkdir graph

In [4]:
import io
from google.colab import auth
from googleapiclient.discovery import build
from googleapiclient.http import (
  MediaFileUpload,
  MediaIoBaseDownload
)

# auth user
auth.authenticate_user()
drive = build('drive', 'v3')

# get parent folder
res = drive.files().list(
      q=f'name="online-search" and trashed=false',
      spaces='drive',
      fields='files(id)',
  ).execute().get('files')
assert res and res[0], 'Graphs not found!'
folder = res[0].get('id')

# get graphs
retrained = drive.files().list(q=f'"{folder}" in parents and trashed=false').execute().get('files')
assert len(retrained) > 0, 'No graph in folder!'

# download
for file in retrained:
  name = file.get('name')
  path = f'./graph/{name}'
  request = drive.files().get_media(fileId=file.get('id'))
  fh = io.FileIO(path, 'wb')
  downloader = MediaIoBaseDownload(fh, request)
  done = False
  while done is False:
    print(f'Downloading {name}')
    status, done = downloader.next_chunk()
  print(f'Complete')

Complete
Complete


#### Set Image

In [0]:
IMAGE = "http://www.artnet.com/WebServices/images/ll00006lldCqVJFgVeECfDrCWvaHBOcEUYF/lucian-freud-portrait-on-a-white-cover.jpg"

#### Download Image

In [6]:
!wget $IMAGE

--2019-03-05 05:10:58--  http://www.artnet.com/WebServices/images/ll00006lldCqVJFgVeECfDrCWvaHBOcEUYF/lucian-freud-portrait-on-a-white-cover.jpg
Resolving www.artnet.com (www.artnet.com)... 104.17.235.7, 104.17.236.7
Connecting to www.artnet.com (www.artnet.com)|104.17.235.7|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 77310 (75K) [image/jpeg]
Saving to: ‘lucian-freud-portrait-on-a-white-cover.jpg’

lucian-freud-portra 100%[===================>]  75.50K  --.-KB/s    in 0.02s   

2019-03-05 05:10:58 (4.27 MB/s) - ‘lucian-freud-portrait-on-a-white-cover.jpg’ saved [77310/77310]



In [0]:
import os
files = os.listdir()
for file in files:
  if file in IMAGE:
    LOCAL_IMAGE = file
    break

#### Run

In [8]:
!python ./tfp2/label_image.py\
  --graph=./graph/retrained_graph_2018-09-29-14-18-37.pb\
  --labels=./graph/retrained_labels_2018-09-29-14-18-37.txt\
  --image=$LOCAL_IMAGE

2019-03-05 05:11:02.398096: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2300000000 Hz
2019-03-05 05:11:02.398372: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x1c9ac60 executing computations on platform Host. Devices:
2019-03-05 05:11:02.398409: I tensorflow/compiler/xla/service/service.cc:158]   StreamExecutor device (0): <undefined>, <undefined>
2019-03-05 05:11:02.509497: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2019-03-05 05:11:02.510268: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x1c9bce0 executing computations on platform CUDA. Devices:
2019-03-05 05:11:02.510328: I tensorflow/compiler/xla/service/service.cc:158]   StreamExecutor device (0): Tesla K80, Compute Capability 3.7
2019-03-05 05:11:02.510776: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1433] Found d